In [ ]:
!nvcc --version☻
!nvidia-smi
import torch
torch.__version__

nvcc fatal   : Unknown option '--version☻'
Sun May 19 15:49:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+------------------------

'2.2.1+cu121'

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'

  Cloning https://github.com/facebookresearch/detectron2.git (to revision 5aeb252b194b93dc2879b4ac34bc51a31b5aee13) to /tmp/pip-req-build-bg741juh
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-bg741juh
  Running command git rev-parse -q --verify 'sha^5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Running command git checkout -q 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
  Preparing metadata (setup.py) ... done


In [ ]:
%cd /content/
!git clone https://github.com/DomMcOyle/TACO-expl.git
%cd /content/TACO-expl
!git checkout maskdino
!git pull origin maskdino
%cd /content/TACO-expl/MaskDINO
!pip install -r requirements.txt
%cd /content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
!sh make.sh

/content
fatal: destination path 'TACO-expl' already exists and is not an empty directory.
/content/TACO-expl
Already on 'maskdino'
Your branch is up to date with 'origin/maskdino'.
From https://github.com/DomMcOyle/TACO-expl
 * branch            maskdino   -> FETCH_HEAD
Already up to date.
/content/TACO-expl/MaskDINO
/content/TACO-expl/MaskDINO/maskdino/modeling/pixel_decoder/ops
running build
running build_py
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:415: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(c

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive/", force_remount = True)

Mounted at /content/MyDrive/


In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.2 ; cuda:  cu121
detectron2: 0.6


In [ ]:
%cd /content/
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import sys
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

sys.path.append('/content/TACO-expl/MaskDINO/maskdino/')
from config import add_maskdino_config
%cd /content/TACO-expl/MaskDINO/maskdino/
from detectron2.projects.deeplab import add_deeplab_config
from maskdino import MaskDINO
%cd /content/

/content
/content/TACO-expl/MaskDINO/maskdino
/content


In [ ]:
sys.path.append("/content/TACO-expl/MaskDINO")
!cd /content/TACO-expl/MaskDINO/
cfg_biggest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/swin/maskdino_R50_bs16_50ep_4s_dowsample1_2048.yaml'
model_biggest_weights_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth'

cfg_smallest_path = '/content/TACO-expl/MaskDINO/configs/coco/instance-segmentation/maskdino_R50_bs16_50ep_3s.yaml'
model_smallest_weight_url = 'https://github.com/IDEA-Research/detrex-storage/releases/download/maskdino-v0.1.0/maskdino_r50_50ep_300q_hid1024_3sd1_instance_maskenhanced_mask46.1ap_box51.5ap.pth'

In [ ]:
def setup_cfg(cfg_path, weights_url):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_maskdino_config(cfg)
    cfg.merge_from_file(cfg_path)
    cfg.MODEL.WEIGHTS = weights_url
    cfg.freeze()
    return cfg

cfg_alt_dino = setup_cfg(cfg_smallest_path, model_smallest_weight_url)
cfg_modified = cfg_alt_dino.copy()
#print(cfg_modified["MODEL"])

In [ ]:
%cd /content/TACO-expl/
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

import os.path
import json
import argparse
import numpy as np
import random
import datetime as dt
import copy
import torch.nn as nn
from sklearn.model_selection import train_test_split
from pathlib import Path

import torch
import math
import torch.utils.data
from torch.utils.data import DataLoader
from pycocotools import mask as coco_mask

/content/TACO-expl


In [ ]:
DEFAULT_RANDOM_SEED = 42
# basic random seed
def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
# torch random seed
def seedTorch(seed=DEFAULT_RANDOM_SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# combine
def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)
    seedTorch(seed)

seedEverything()

In [ ]:
keep_categories = ["Bottle", "Bottle cap", "Can", "Cigarette", "Cup",
                   "Lid", "Plastic bag & wrapper", "Pop tab", "Straw"]

def create_map(original, keep_supercategories):
  class_map = {}
  for cat in original:
    if cat["supercategory"] in keep_supercategories:
      class_map[cat["name"]] = cat["supercategory"]
    else:
      class_map[cat["name"]] = "Other"
  return class_map

def replace_dataset_classes(dataset, class_map):
      """ Replaces classes of dataset based on a dictionary"""
      class_new_names = list(set(class_map.values()))
      class_new_names.sort()
      class_originals = copy.deepcopy(dataset['categories'])
      dataset['categories'] = []
      class_ids_map = {}  # map from old id to new id

      # Assign background id 0
      has_background = False
      if 'Background' in class_new_names:
          if class_new_names.index('Background') != 0:
              class_new_names.remove('Background')
              class_new_names.insert(0, 'Background')
          has_background = True

      # Replace categories
      for id_new, class_new_name in enumerate(class_new_names):
          # Make sure id:0 is reserved for background
          id_rectified = id_new
          if not has_background:
              id_rectified += 1

          category = {
              'supercategory': '',
              'id': id_rectified,  # Background has id=0
              'name': class_new_name,
          }
          dataset['categories'].append(category)
          # Map class names
          for class_original in class_originals:
              if class_map[class_original['name']] == class_new_name:
                  class_ids_map[class_original['id']] = id_rectified

      # Update annotations category id tag
      for ann in dataset['annotations']:
          ann['category_id'] = class_ids_map[ann['category_id']]

In [ ]:
with open("/content/TACO-expl/data/annotations.json", "r") as f: #"/content/TACO/data/annotations.json"
    dataset = json.loads(f.read())

class_map = create_map(dataset["categories"], keep_categories)
replace_dataset_classes(dataset, class_map)
dataset["categories"]

[{'supercategory': '', 'id': 1, 'name': 'Bottle'},
 {'supercategory': '', 'id': 2, 'name': 'Bottle cap'},
 {'supercategory': '', 'id': 3, 'name': 'Can'},
 {'supercategory': '', 'id': 4, 'name': 'Cigarette'},
 {'supercategory': '', 'id': 5, 'name': 'Cup'},
 {'supercategory': '', 'id': 6, 'name': 'Lid'},
 {'supercategory': '', 'id': 7, 'name': 'Other'},
 {'supercategory': '', 'id': 8, 'name': 'Plastic bag & wrapper'},
 {'supercategory': '', 'id': 9, 'name': 'Pop tab'},
 {'supercategory': '', 'id': 10, 'name': 'Straw'}]

In [ ]:
classes = [elem["name"] for elem in dataset["categories"]]
print(classes)

['Bottle', 'Bottle cap', 'Can', 'Cigarette', 'Cup', 'Lid', 'Other', 'Plastic bag & wrapper', 'Pop tab', 'Straw']


In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog

help(register_coco_instances)

Help on function register_coco_instances in module detectron2.data.datasets.coco:

register_coco_instances(name, metadata, json_file, image_root)
    Register a dataset in COCO's json annotation format for
    instance detection, instance segmentation and keypoint detection.
    (i.e., Type 1 and 2 in http://cocodataset.org/#format-data.
    `instances*.json` and `person_keypoints*.json` in the dataset).
    
    This is an example of how to register a new dataset.
    You can do something similar to this function, to register new datasets.
    
    Args:
        name (str): the name that identifies a dataset, e.g. "coco_2014_train".
        metadata (dict): extra metadata associated with this dataset.  You can
            leave it as an empty dict.
        json_file (str): path to the json instance annotation file.
        image_root (str or path-like): directory which contains all the images.



In [ ]:
train_annotation_file = '/content/TACO-expl/data/annotations_off_0_train.json'
val_annotation_file = '/content/TACO-expl/data/annotations_off_0_resval.json'

img_dir_train = '/content/MyDrive/MyDrive/res_official/rot_train/'
img_dir_val = '/content/MyDrive/MyDrive/res_official/res_val/'

register_coco_instances("TACO_train", {}, train_annotation_file, img_dir_train)
MetadataCatalog.get("TACO_train").set(thing_classes = classes)
dataset_dicts_train = DatasetCatalog.get("TACO_train")

register_coco_instances("TACO_val", {}, val_annotation_file, img_dir_val)
MetadataCatalog.get("TACO_val").set(thing_classes = classes)
dataset_dicts_val = DatasetCatalog.get("TACO_val")

[05/19 15:50:59 d2.data.datasets.coco]: Loaded 1200 images in COCO format from /content/TACO-expl/data/annotations_off_0_train.json
[05/19 15:51:00 d2.data.datasets.coco]: Loaded 150 images in COCO format from /content/TACO-expl/data/annotations_off_0_resval.json


In [ ]:
import json

with open(val_annotation_file, "r") as f:
  val_annotations = json.load(f)

print(val_annotations.keys())

dict_keys(['info', 'scene_annotations', 'licenses', 'categories', 'scene_categories', 'images', 'annotations'])


In [ ]:
first_img = val_annotations["images"][0]
first_ann = val_annotations["annotations"][0]

print(first_img)
print(first_ann)

{'id': 899, 'width': 1067, 'height': 800, 'file_name': 'batch_4/000023.JPG', 'license': None, 'flickr_url': 'https://farm66.staticflickr.com/65535/47803892332_5218b74150_o.png', 'coco_url': None, 'date_captured': None, 'flickr_640_url': 'https://farm66.staticflickr.com/65535/47803892332_f552fb65c7_z.jpg'}
{'id': 2802, 'image_id': 899, 'category_id': 2, 'segmentation': {'counts': [541232, 2, 798, 7, 792, 10, 789, 12, 787, 13, 786, 15, 785, 15, 783, 18, 779, 22, 776, 24, 774, 27, 772, 28, 771, 29, 770, 31, 768, 32, 767, 33, 766, 34, 765, 36, 763, 37, 763, 37, 763, 37, 763, 37, 763, 37, 763, 37, 763, 38, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 761, 39, 762, 38, 762, 38, 762, 38, 762, 37, 764, 36, 764, 36, 765, 35, 765, 34, 767, 33, 768, 32, 769, 30, 771, 29, 772, 28, 774, 25, 776, 5, 1, 18, 783, 17, 783, 16, 785, 15, 786, 13, 788, 11, 790, 9, 793, 6, 798, 1, 2

In [ ]:
# run this cell if you want a fresh config file to modify
train_cfg = cfg_alt_dino.clone()
with open("/content/TACO-expl/MaskDINO/configs/taco/taco_train.yaml", "w") as f:
  f.write(train_cfg.dump())

In [ ]:
!mkdir /content/TACO-expl/maskdino_config/
!mkdir /content/output/
!mkdir /content/output/chkpt/
#!cp /content/MyDrive/MyDrive/maskdino_models/model_0000478.pth /content/output/chkpt/

In [ ]:
train_cfg_loaded = get_cfg()
train_cfg_loaded.set_new_allowed(True)
train_cfg_loaded.merge_from_file("/content/TACO-expl/maskdino_config/taco_train_maskdino.yaml")
print(train_cfg_loaded.MODEL.BACKBONE.FREEZE_AT)
print(train_cfg_loaded.SOLVER.IMS_PER_BATCH)
print(train_cfg_loaded.INPUT.MIN_SIZE_TRAIN)
print(train_cfg_loaded.DATASETS.PRECOMPUTED_PROPOSAL_TOPK_TRAIN)
print(train_cfg_loaded.TEST.AUG.ENABLED)
print(train_cfg_loaded.INPUT.MASK_FORMAT)
print(train_cfg_loaded.Default_loading)

4
2
(480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800)
100
False
polygon
False


In [ ]:
%cd /content/TACO-expl/MaskDINO/
from train_net import Trainer

/content/TACO-expl/MaskDINO


In [ ]:
!python -m pip install numpy==1.23.1
import numpy as np
np.bool = np.bool_

In [ ]:
%cd /content/output/chkpt/

/content/output/chkpt


In [ ]:
trainer = Trainer(train_cfg_loaded)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

In [ ]:
#trainer.build_hooks()
trainer.resume_or_load(resume = True)  # load last checkpoint or MODEL.WEIGHTS
trainer.train()

WARNING [05/19 15:51:40 d2.checkpoint.c2_model_loading]: Shape of criterion.empty_weight in checkpoint is torch.Size([81]), while shape of criterion.empty_weight in model is torch.Size([11]).
WARNING [05/19 15:51:40 d2.checkpoint.c2_model_loading]: criterion.empty_weight will not be loaded. Please double check and see if this is desired.
WARNING [05/19 15:51:40 d2.checkpoint.c2_model_loading]: Shape of sem_seg_head.predictor.class_embed.bias in checkpoint is torch.Size([80]), while shape of sem_seg_head.predictor.class_embed.bias in model is torch.Size([10]).
WARNING [05/19 15:51:40 d2.checkpoint.c2_model_loading]: sem_seg_head.predictor.class_embed.bias will not be loaded. Please double check and see if this is desired.
WARNING [05/19 15:51:40 d2.checkpoint.c2_model_loading]: Shape of sem_seg_head.predictor.class_embed.weight in checkpoint is torch.Size([80, 256]), while shape of sem_seg_head.predictor.class_embed.weight in model is torch.Size([10, 256]).
WARNING [05/19 15:51:40 d2.ch

criterion.empty_weight
sem_seg_head.predictor.class_embed.{bias, weight}
sem_seg_head.predictor.label_enc.weight


[05/19 15:51:40 d2.engine.train_loop]: Starting training from iteration 0


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learnin

[05/19 15:52:19 d2.utils.events]:  eta: 2:25:47  iter: 19  total_loss: 2.105e+04  loss_ce: 1888  loss_mask: 0.1332  loss_dice: 2.025  loss_bbox: 0.5632  loss_giou: 1.578  loss_ce_dn: 14.28  loss_mask_dn: 0.1457  loss_dice_dn: 1.74  loss_bbox_dn: 0.219  loss_giou_dn: 0.8272  loss_ce_0: 1991  loss_mask_0: 0.4588  loss_dice_0: 2.836  loss_bbox_0: 0.761  loss_giou_0: 1.993  loss_ce_dn_0: 10.1  loss_mask_dn_0: 0.7581  loss_dice_dn_0: 4.397  loss_bbox_dn_0: 0.2388  loss_giou_dn_0: 0.854  loss_ce_1: 2218  loss_mask_1: 0.4049  loss_dice_1: 2.423  loss_bbox_1: 0.7104  loss_giou_1: 1.821  loss_ce_dn_1: 13.91  loss_mask_dn_1: 0.09831  loss_dice_dn_1: 1.635  loss_bbox_dn_1: 0.2566  loss_giou_dn_1: 0.9161  loss_ce_2: 1957  loss_mask_2: 0.2873  loss_dice_2: 2.296  loss_bbox_2: 0.7366  loss_giou_2: 1.84  loss_ce_dn_2: 13.79  loss_mask_dn_2: 0.08488  loss_dice_dn_2: 1.407  loss_bbox_dn_2: 0.2142  loss_giou_dn_2: 0.7682  loss_ce_3: 1959  loss_mask_3: 0.2512  loss_dice_3: 2.382  loss_bbox_3: 0.7082  los

...

[05/19 18:43:34 d2.evaluation.evaluator]: Total inference time: 0:01:54.979382 (0.792961 s / iter per device, on 1 devices)
[05/19 18:43:34 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:41 (0.284413 s / iter per device, on 1 devices)
[05/19 18:43:34 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/19 18:43:34 d2.evaluation.coco_evaluation]: Saving results to /content/output/chkpt/inference/coco_instances_results.json
[05/19 18:43:34 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
[05/19 18:43:34 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/19 18:43:35 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.22 seconds.
[05/19 18:43:35 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[05/19 18:43:35 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.10 seconds.
 

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/19 18:50:32 d2.evaluation.evaluator]: Inference done 11/150. Dataloading: 0.0048 s/iter. Inference: 0.2940 s/iter. Eval: 0.5429 s/iter. Total: 0.8417 s/iter. ETA=0:01:57
[05/19 18:50:38 d2.evaluation.evaluator]: Inference done 18/150. Dataloading: 0.0034 s/iter. Inference: 0.2782 s/iter. Eval: 0.4992 s/iter. Total: 0.7810 s/iter. ETA=0:01:43
[05/19 18:50:43 d2.evaluation.evaluator]: Inference done 24/150. Dataloading: 0.0041 s/iter. Inference: 0.2833 s/iter. Eval: 0.5198 s/iter. Total: 0.8075 s/iter. ETA=0:01:41
[05/19 18:50:48 d2.evaluation.evaluator]: Inference done 30/150. Dataloading: 0.0058 s/iter. Inference: 0.2916 s/iter. Eval: 0.5450 s/iter. Total: 0.8428 s/iter. ETA=0:01:41
[05/19 18:50:54 d2.evaluation.evaluator]: Inference done 37/150. Dataloading: 0.0051 s/iter. Inference: 0.2875 s/iter. Eval: 0.5267 s/iter. Total: 0.8196 s/iter. ETA=0:01:32
[05/19 18:50:59 d2.evaluation.evaluator]: Inference done 44/150. Dataloading: 0.0046 s/iter. Inference: 0.2865 s/iter. Eval: 0.526

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[05/19 18:52:24 d2.evaluation.evaluator]: Total inference time: 0:01:57.129134 (0.807787 s / iter per device, on 1 devices)
[05/19 18:52:24 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:41 (0.284546 s / iter per device, on 1 devices)
[05/19 18:52:25 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/19 18:52:25 d2.evaluation.coco_evaluation]: Saving results to /content/output/chkpt/inference/coco_instances_results.json
[05/19 18:52:25 d2.evaluation.coco_evaluation]: Evaluating predictions with unofficial COCO API...
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!
[05/19 18:52:25 d2.evaluation.fast_eval_api]: Evaluate annotation type *bbox*
[05/19 18:52:25 d2.evaluation.fast_eval_api]: COCOeval_opt.evaluate() finished in 0.22 seconds.
[05/19 18:52:25 d2.evaluation.fast_eval_api]: Accumulating evaluation results...
[05/19 18:52:26 d2.evaluation.fast_eval_api]: COCOeval_opt.accumulate() finished in 0.07 seconds.
 

In [ ]:
os.listdir("/content/output/chkpt/")

['model_0001915.pth',
 'model_0003831.pth',
 'model_final.pth',
 'events.out.tfevents.1715969610.7688afa66bed.3673.0',
 'model_0002873.pth',
 'model_0001436.pth',
 'model_0005268.pth',
 'model_0003352.pth',
 'model_0002394.pth',
 'metrics.json',
 'inference',
 'model_0000957.pth',
 'model_0004789.pth',
 'model_0000478.pth',
 'events.out.tfevents.1715971997.7688afa66bed.12409.0',
 'model_0005747.pth',
 'last_checkpoint',
 'model_0004310.pth']

In [ ]:
from google.colab import files
files.download("/content/output/chkpt/model_final.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/output/chkpt/model_0005747.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/output/chkpt/model_0005268.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/output/chkpt/last_checkpoint")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run this if you want to check how many trainable parameters we have

In [ ]:
from detectron2.modeling import build_model
model = build_model(train_cfg_loaded)
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_trainable_params)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':